### VERN AdU Waste MobileNetV3 Small Model

In [ ]:
# Install necessary libraries
!pip install --upgrade pip -q
!pip install tensorflow==2.16.1 tf-keras==2.16.0 scikit-learn==1.4.2 matplotlib seaborn pillow -q
print("Libraries installation completed.")

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
import numpy as np
import os
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageFile
from google.colab import drive, files

# Allow processing of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True
print("Libraries imported successfully.")

In [ ]:
# Mount Google Drive to access the dataset
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")

In [ ]:
# Paths for dataset and output directories
google_drive_path = '/content/drive/MyDrive/CSRP/dataSet'
merged_dataset_dir = '/content/merged_dataset'
os.makedirs(merged_dataset_dir, exist_ok=True)

# Function to copy and validate images, with enhanced error handling
def copy_and_validate_images(source_dir, subcategories, category, target_dir):
    try:
        target_category_dir = os.path.join(target_dir, category)
        os.makedirs(target_category_dir, exist_ok=True)
        total_images_in_category = 0

        for subcategory in subcategories:
            subcategory_dir = os.path.join(source_dir, category, subcategory)
            image_files = os.listdir(subcategory_dir)
            total_images = len(image_files)
            total_images_in_category += total_images
            print(f"Processing {total_images} images in '{subcategory}' under '{category}'.")

            for img_file in image_files:
                source_file_path = os.path.join(subcategory_dir, img_file)
                target_file_path = os.path.join(target_category_dir, img_file)
                if os.path.exists(target_file_path):
                    continue

                try:
                    with Image.open(source_file_path) as img:
                        if img.mode in ('P', 'PA'):
                            img = img.convert('RGBA')
                        if img.mode == 'RGBA':
                            img = img.convert('RGB')
                        elif img.mode == 'LA':
                            img = img.convert('L')
                        if img.mode not in ('RGB', 'L'):
                            img = img.convert('RGB')
                        img.save(target_file_path)
                except (IOError, OSError) as e:
                    print(f"Skipped image '{img_file}' due to: {e}")

            print(f"Completed '{subcategory}' in '{category}'.")
        print(f"Total images processed in '{category}': {total_images_in_category}\n")
    except Exception as e:
        print(f"Error processing '{category}': {e}")

In [ ]:
# Define subcategories for each waste category
biodegradable_subcategories = ['food_waste', 'leaf_waste', 'paper_waste', 'wood_waste']
non_biodegradable_subcategories = ['ewaste', 'metal_cans', 'other', 'plastic_bags', 'plastic_bottles']
recyclable_subcategories = ['aluminum', 'carton', 'foam_box', 'milk_box', 'other', 'paper', 'paper_cup', 'plastic', 'plastic_cup']

# Merge and validate the dataset
copy_and_validate_images(google_drive_path, biodegradable_subcategories, 'biodegradable', merged_dataset_dir)
copy_and_validate_images(google_drive_path, non_biodegradable_subcategories, 'non_biodegradable', merged_dataset_dir)
copy_and_validate_images(google_drive_path, recyclable_subcategories, 'recyclable', merged_dataset_dir)
print("Dataset merging completed.")

In [ ]:
# Data generators for training, validation, and testing
try:
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2],
        fill_mode='nearest',
        validation_split=0.2
    )

    # Validation and test data generators (rescaling only)
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Create training data generator
    train_generator = train_datagen.flow_from_directory(
        merged_dataset_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training',
        seed=42
    )
    print(f"Training data generator created with {train_generator.samples} samples.")

    # Create validation data generator
    validation_generator = train_datagen.flow_from_directory(
        merged_dataset_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='validation',
        seed=42
    )
    print(f"Validation data generator created with {validation_generator.samples} samples.")

    # Create test data generator
    test_generator = test_datagen.flow_from_directory(
        merged_dataset_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )
    print(f"Test data generator created with {test_generator.samples} samples.")

except Exception as e:
    print(f"Error in creating data generators: {e}")

In [ ]:
# Compute class weights for handling class imbalance
try:
    class_counts = Counter(train_generator.classes)
    total_samples = len(train_generator.classes)
    print(f"Class distribution in training data: {class_counts}")
    print(f"Total training samples: {total_samples}")

    # Compute and normalize class weights
    class_weights = {
        class_id: np.log(total_samples / count)
        for class_id, count in class_counts.items()
    }
    max_weight = max(class_weights.values())
    class_weights = {class_id: weight / max_weight for class_id, weight in class_weights.items()}
    print(f"Normalized class weights: {class_weights}")

except Exception as e:
    print(f"Class weight calculation error: {e}")

In [ ]:
# Build and compile the model with staged fine-tuning and exponential decay learning rate
try:
    base_model = MobileNetV3Small(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

    # Freeze 60% of layers for initial training (Stage 1)
    for layer in base_model.layers[:int(0.6 * len(base_model.layers))]:
        layer.trainable = False

    # Add custom layers
    x = GlobalAveragePooling2D()(base_model.output)
    x = BatchNormalization()(x)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(1e-5))(x)
    x = Dropout(0.4)(x)
    predictions = Dense(train_generator.num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Exponential decay learning rate schedule
    lr_schedule_stage_1 = ExponentialDecay(
        initial_learning_rate=1e-4, decay_steps=3 * len(train_generator), decay_rate=0.9, staircase=True
    )
    model.compile(optimizer=Adam(learning_rate=lr_schedule_stage_1), loss='categorical_crossentropy', metrics=['accuracy'])
    print("Model compiled for Stage 1.")

except Exception as e:
    print(f"Model building error: {e}")

In [ ]:
# Train the model in Stage 1
try:
    history_stage_1 = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        class_weight=class_weights,
        callbacks=[
            ModelCheckpoint('best_model_stage_1.keras', monitor='val_accuracy', save_best_only=True, mode='max'),
            EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        ]
    )
    print("Stage 1 training completed.")

except Exception as e:
    print(f"Stage 1 training error: {e}")

In [ ]:
# Fine-tune more layers (Stage 2)
try:
    for layer in base_model.layers[:int(0.3 * len(base_model.layers))]:
        layer.trainable = False
    for layer in base_model.layers[int(0.3 * len(base_model.layers)):]:
        layer.trainable = True

    lr_schedule_stage_2 = ExponentialDecay(
        initial_learning_rate=5e-5, decay_steps=3 * len(train_generator), decay_rate=0.9, staircase=True
    )
    model.compile(optimizer=Adam(learning_rate=lr_schedule_stage_2), loss='categorical_crossentropy', metrics=['accuracy'])
    print("Model recompiled for Stage 2.")

    history_stage_2 = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        class_weight=class_weights,
        callbacks=[
            ModelCheckpoint('best_model_stage_2.keras', monitor='val_accuracy', save_best_only=True, mode='max'),
            EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        ]
    )
    print("Stage 2 fine-tuning completed.")

except Exception as e:
    print(f"Stage 2 fine-tuning error: {e}")

In [ ]:
# Fine-tune all layers (Stage 3)
try:
    for layer in base_model.layers:
        layer.trainable = True

    lr_schedule_stage_3 = ExponentialDecay(
        initial_learning_rate=1e-5, decay_steps=3 * len(train_generator), decay_rate=0.9, staircase=True
    )
    model.compile(optimizer=Adam(learning_rate=lr_schedule_stage_3), loss='categorical_crossentropy', metrics=['accuracy'])
    print("Model recompiled for Stage 3.")

    history_stage_3 = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        class_weight=class_weights,
        callbacks=[
            ModelCheckpoint('best_model_stage_3.keras', monitor='val_accuracy', save_best_only=True, mode='max'),
            EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        ]
    )
    print("Stage 3 fine-tuning completed.")

except Exception as e:
    print(f"Stage 3 fine-tuning error: {e}")

In [ ]:
# Evaluate the model and compute metrics
try:
    test_loss, test_accuracy = model.evaluate(test_generator)
    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")

    # Generate predictions and compute metrics
    y_true = test_generator.classes
    y_pred_probs = model.predict(test_generator)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)

    # Calculate precision, recall, and F1 score
    precision = precision_score(y_true, y_pred_classes, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred_classes, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred_classes, average='weighted', zero_division=0)
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

except Exception as e:
    print(f"Model evaluation error: {e}")

In [ ]:
# Plot training history for all stages
try:
    plt.figure(figsize=(12, 6))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history_stage_1.history['accuracy'], label='Stage 1 Train Acc')
    plt.plot(history_stage_1.history['val_accuracy'], label='Stage 1 Val Acc')
    plt.plot(history_stage_2.history['accuracy'], label='Stage 2 Train Acc')
    plt.plot(history_stage_2.history['val_accuracy'], label='Stage 2 Val Acc')
    plt.plot(history_stage_3.history['accuracy'], label='Stage 3 Train Acc')
    plt.plot(history_stage_3.history['val_accuracy'], label='Stage 3 Val Acc')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='upper left')

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history_stage_1.history['loss'], label='Stage 1 Train Loss')
    plt.plot(history_stage_1.history['val_loss'], label='Stage 1 Val Loss')
    plt.plot(history_stage_2.history['loss'], label='Stage 2 Train Loss')
    plt.plot(history_stage_2.history['val_loss'], label='Stage 2 Val Loss')
    plt.plot(history_stage_3.history['loss'], label='Stage 3 Train Loss')
    plt.plot(history_stage_3.history['val_loss'], label='Stage 3 Val Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')

    plt.tight_layout()
    plt.show()
    print("Training history plotted.")

except Exception as e:
    print(f"Error plotting training history: {e}")

In [ ]:
# Generate and display the confusion matrix
try:
    conf_matrix = confusion_matrix(y_true, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=test_generator.class_indices, yticklabels=test_generator.class_indices)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()
    print("Confusion matrix displayed.")

except Exception as e:
    print(f"Confusion matrix error: {e}")

In [ ]:
# Convert the model to TensorFlow Lite for deployment
try:
    K.clear_session()  # Clear the session to free up resources

    # Export the trained model to a SavedModel format
    model_dir = 'saved_model'
    model.export(model_dir)
    print(f"Model exported successfully to directory: {model_dir}")

    # Convert the exported model to TensorFlow Lite
    converter = tf.lite.TFLiteConverter.from_saved_model(model_dir)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]
    tflite_model = converter.convert()

    # Save the TensorFlow Lite model
    tflite_model_path = 'waste_classifier.tflite'
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)
    print(f"Model converted to TensorFlow Lite and saved as: {tflite_model_path}")

    # Download the TensorFlow Lite model file
    files.download(tflite_model_path)
    print("TensorFlow Lite model downloaded.")

except Exception as e:
    print(f"TensorFlow Lite conversion error: {e}")